In [22]:
# Importar bibliotecas necessárias
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
import joblib
from google.colab import files

# Solicitar o upload do dataset
uploaded = files.upload()

# Ler o dataset (substitua pelo nome do arquivo após o upload)
df = pd.read_csv("dataset_ampliado_200_exemplos_novas_perguntas.csv")

# Definir listas de perguntas para problemas elétricos e mecânicos
problemas_eletricos = [
    "Você notou alguma luz do seu carro que não está funcionando corretamente, como faróis, lanternas ou luzes de freio?",
    "Seu carro está apresentando algum problema de intermitência nas luzes, onde elas piscam ou desligam e ligam aleatoriamente?",
    "Você percebeu alguma mudança na intensidade das luzes do seu carro, como ficarem mais fracas do que o normal?",
    "O seu carro está apresentando algum problema com o ajuste automático dos faróis, onde eles não se ajustam corretamente de acordo com as condições de luminosidade?",
    "As luzes internas do carro estão funcionando de forma irregular?",
    "A luz do painel de bateria está acesa de forma intermitente?",
    "Alguma das luzes de segurança, como a de freio ou cinto, está apresentando problemas?"
]

problemas_mecanicos = [
    "Houve algum vazamento de óleo embaixo do seu carro? É importante verificar se há sinais de vazamentos no motor.",
    "Já foi feita a manutenção da correia dentada ou corrente de distribuição do seu carro dentro do prazo recomendado pelo fabricante?",
    "O seu carro tem apresentado problemas de desempenho ou consumo excessivo de combustível? Isso poderia estar relacionado a problemas nos sensores do motor.",
    "Seu carro está apresentando falhas de partida ou dificuldades para manter o funcionamento do motor? Isso pode ser um problema no sistema de injeção de combustível.",
    "O carro tem emitido algum tipo de fumaça pelo escapamento?",
    "O câmbio do carro está mais difícil de manusear do que o normal?",
    "Há algum ruído incomum vindo da suspensão ao passar por lombadas?"
]

# Simplificar as respostas para todas as perguntas
def simplify_response(response):
    return 1 if response == "Sim" else 0

for coluna in problemas_eletricos + problemas_mecanicos:
    df[coluna] = df[coluna].apply(simplify_response)

# Criar novas features baseadas nas contagens de problemas
df['contagem_problemas_eletricos'] = df[problemas_eletricos].sum(axis=1)
df['contagem_problemas_mecanicos'] = df[problemas_mecanicos].sum(axis=1)
df['intensidade_total_problemas'] = df['contagem_problemas_eletricos'] + df['contagem_problemas_mecanicos']
df['diferença_problemas'] = abs(df['contagem_problemas_eletricos'] - df['contagem_problemas_mecanicos'])

# Selecionar as features e a variável de saída para a regressão
X = df[['contagem_problemas_eletricos', 'contagem_problemas_mecanicos']]
y = df['intensidade_total_problemas']

# Dividir os dados em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Definir o modelo de regressão
regressor = RandomForestRegressor(n_estimators=100, random_state=42)
regressor.fit(X_train, y_train)

# Fazer previsões e avaliar o modelo
y_train_pred = regressor.predict(X_train)
y_test_pred = regressor.predict(X_test)
train_mse = mean_squared_error(y_train, y_train_pred)
test_mse = mean_squared_error(y_test, y_test_pred)
train_r2 = r2_score(y_train, y_train_pred)
test_r2 = r2_score(y_test, y_test_pred)

print(f"Mean Squared Error (treino): {train_mse:.4f}")
print(f"Mean Squared Error (teste): {test_mse:.4f}")
print(f"R^2 Score (treino): {train_r2:.4f}")
print(f"R^2 Score (teste): {test_r2:.4f}")

# Definir limiares para decidir as classes a partir das previsões de regressão
limiar_intensidade = y_train.median()
pred_classes = ["Mecânico" if pred > limiar_intensidade else "Eletricidade Geral" for pred in y_test_pred]

# Exibir as previsões finais
print(pred_classes[:10])  # Exibindo as primeiras previsões para conferência

# Salvar o modelo de regressão treinado
joblib.dump(regressor, "model_regressor_intensidade.pkl")


Saving dataset_completo.csv to dataset_completo (2).csv
Mean Squared Error (treino): 0.0035
Mean Squared Error (teste): 0.1597
R^2 Score (treino): 0.9987
R^2 Score (teste): 0.9641
['Mecânico', 'Eletricidade Geral', 'Mecânico', 'Eletricidade Geral', 'Eletricidade Geral', 'Mecânico', 'Mecânico', 'Mecânico', 'Eletricidade Geral', 'Mecânico']


['model_regressor_intensidade.pkl']